In [ ]:
import vtk
from cfdmod.api.vtk.write_vtk import read_polydata, write_polydata
import vtk

import pathlib
from typing import Literal

def envelope_vtks(polydatas: list[vtk.vtkPolyData], scalar: str, stats:Literal["min", "max"]) -> vtk.vtkPolyData:
    if stats not in {"min", "max"}: 
        raise ValueError("stats must be 'min' or 'max'")
    if(len(polydatas) == 0):
        raise ValueError("Empty input list")

    # Start with a deep copy of the first polydata
    envelope_polydata = vtk.vtkPolyData()
    envelope_polydata.DeepCopy(polydatas[0])

    # Extract the scalar array from the envelope copy (this will be updated)
    envelope_array = envelope_polydata.GetCellData().GetArray(scalar)
    n_cells = envelope_polydata.GetNumberOfCells()

    # Loop through all other polydata
    for polydata in polydatas[1:]:
        other_array = polydata.GetCellData().GetArray(scalar)
        for i in range(n_cells):
            current_val = envelope_array.GetValue(i)
            new_val = other_array.GetValue(i)
            if stats == "max":
                envelope_array.SetValue(i, max(current_val, new_val))
            elif stats == "min":
                envelope_array.SetValue(i, min(current_val, new_val))

    envelope_array.Modified()
    return envelope_polydata

base_path = pathlib.Path("/mnt/disk01/prd-eng/consulting/048_TelmecCajamar/deliverables/new_diagrams")
directions = [f"{i:03d}" for i in range(0, 30, 360)]
filenames = [base_path / f"{d}.vtp" for d in directions]
polydatas = [read_polydata(p) for p in filenames]

envelope_min = envelope_vtks(polydatas, "min", "min")
envelope_max = envelope_vtks(polydatas, "max", "max")

write_polydata(base_path / "envelope_max.vtp",envelope_max)
write_polydata(base_path / "envelope_min.vtp",envelope_min)
